# トレンドレコメンド（仮）

## 概要

- 世間で話題になっているトレンドを抽出し、トレンドに関連した商品を自動抽出する
- 使用したAPI （TwitterAPI, NewsAPI, 楽天API）

## 処理の流れ

1. 以下のいずれかの方法により、トレンドワードを取得する
- GoogleトレンドのWebページをクローリング・スクレイピングし、Google検索で急上昇中のワードを取得する（https://trends.google.co.jp/trends/trendingsearches/realtime?geo=JP&category=all）
- TwitterAPIを使用し、Twitterでトレンドになっているワードを取得する
- NewsAPIを使用し、国内ニュースでトレンドになっているニュースを取得する（これはワードではなく、ニュース記事を取得）
<br>
※ 各処理で必要となる処理は関数化済みだが、今回はGoogleトレンドのパターンで実験する


2. トレンドワードに関連するニュース記事をNewsAPIを使用して取得する
- 1で取得したトレンドワードでニュース記事を検索する
- ニュース記事をタイトルと概要を取得する（本文取得はAPIではできない）

3. トレンドワードに関連するツイートをTwitterAPIを使用して取得する
- 取得したトレンドワードでツイートを検索する

4. トレンドワード・ニュース記事（複数）・ツイート（10件）を一つのテキストにまとめる
5. 楽天APIを使用し、楽天ブックスで販売する書籍のタイトル・著者・概要を取得する
- 一度に楽天APIで取得できる商品数には制限がある為、事前にAPIを一日叩き続けて取得できた10万件近い書籍のデータを使用する
- 書籍データはpandasのデータとして保持し、pickleファイル化しておく（実際はDBに保存しておきたい）

6. 5で取得した書籍情報（タイトル・著者・概要）を一つのテキストにまとめる

7. 4と6をMecabで形態素解析し、名詞だけを取得する

8. TFIDFのベクトル化して、トレンドに関連するテキスト（トレンドワード・ニュース記事・ツイート）に一番類似する商品をCOS類似度を算出し求める

## 以下、処理内容

In [1]:
# 必要なライブラリをインストール（AWSで環境をゼロから作り直す時必要）
# !pip install newsapi-python
# !pip install twitter
# !pip install selenium
# !pip install feedparser

### 前処理

In [2]:
# 各APIで必要なライブラリをインポート
from newsapi import NewsApiClient
from twitter import *

In [3]:
# その他、必要なライブラリをインポート
import datetime
import re
from selenium import webdriver
import time
import feedparser
import random

In [4]:
# API_KEYの設定ファイルを読み込み
# ※ APIを使用する為に必要となるキーとパスをまとめた設定ファイルを別途作成し、それを呼び出している
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

### news_api

In [5]:
# news_apiから、検索したキーワードに関連するニュース情報を返す関数
def keyword_news_search(\
                         api_key = config['NEWS_API']['KEY'],\
                         search_word = '', \
                         start_date = datetime.datetime.now().strftime('%Y-%m-%d'),\
                         end_date = datetime.datetime.now().strftime('%Y-%m-%d'), \
                         sort_type = 'popularity'\
                        ):
    
    # news_apiを初期化
    news_api_search = NewsApiClient(api_key=config['NEWS_API']['KEY'])
    
    # search_wordで指定したキーワードに関するニュースを検索する
    result_search_news = news_api_search.get_everything(qintitle=search_word, from_param=start_date ,to=end_date, sort_by=sort_type)
    
    return result_search_news

In [7]:
# news_apiから、トップニュース情報を返す関数
def headlines_news(\
                         api_key = config['NEWS_API']['KEY'],\
                         target_category = 'entertainment', \
                         target_country = 'jp'
                        ):
    
    # news_apiを初期化
    news_api_headlines = NewsApiClient(api_key=config['NEWS_API']['KEY'])
    
    # 指定したオプションに従ったトップニュースを検索する     
    # Possible options: business entertainment general health science sports technology . 
    # Note: you can't mix this param with the sources param.
    result_headlines_news = news_api_headlines.get_top_headlines(category=target_category, country=target_country)
    
    return result_headlines_news

### twitter_api

In [8]:
# twitter_apiから、ツイートトレンドの単語を返す関数
def twitter_trends_search(\
                          CK = config['TWITTER_API']['CONSUMER_KEY'],\
                          CS = config['TWITTER_API']['CONSUMER_SECRET'],\
                          AT = config['TWITTER_API']['ACCESS_TOKEN'],\
                          AS = config['TWITTER_API']['ACCESS_TOKEN_SECRET'],\
                          area_id = 23424856\
                         ):
    
     # twitter_apiを初期化
    twitter_api_trends = Twitter(auth = OAuth(AT,AS,CK,CS))
        
     # 指定したエリアのツイッタートレンドを検索する
    twitter_trends_results = twitter_api_trends.trends.place(_id = area_id)
                    
    return twitter_trends_results

In [9]:
# twitter_apiから、検索したキーワードに関連するツイートを返す関数
def twitter_search(\
                          CK = config['TWITTER_API']['CONSUMER_KEY'],\
                          CS = config['TWITTER_API']['CONSUMER_SECRET'],\
                          AT = config['TWITTER_API']['ACCESS_TOKEN'],\
                          AS = config['TWITTER_API']['ACCESS_TOKEN_SECRET'],\
                          search_keyword = 'twitter',\
                          search_lang = 'ja',\
                          search_type = 'mixed',\
                          search_count = 100\
                         ):
    
     # twitter_apiを初期化
    twitter_api_search = Twitter(auth = OAuth(AT,AS,CK,CS))
        
     # 指定したエリアのツイッタートレンドを検索する
    search_results = twitter_api_search.search.tweets(q=search_keyword, lang=search_lang, result_type=search_type, count=search_count)
                    
    return search_results

### Googleトレンド（クローリング・スクレイピング）

In [10]:
def google_trends_search(\
                        base_url = 'https://trends.google.co.jp/trends/trendingsearches/realtime',\
                        search_geo = 'JP',\
                        search_category = 'all',
                        ):
    
    # 検索するURLを作成する
    url = '{i}?geo={j}&category={k}'.format(i=base_url, j=search_geo, k=search_category)
    
    # ブラウザを開く
    driver = webdriver.Chrome()
    
    # Googleトレンドの検索TOP画面を開く （表示サイズも指定しておく）
    driver.get(url)
    driver.set_window_size(2500,1000)
    
    # 2〜4秒間、ページが開かれるのを待つ
    time.sleep(random.randint(2,4))
    
    # TOP画面より、必要な要素のデータを取得する
    trends_keywords = driver.find_elements_by_class_name('title')
    summary_text = driver.find_elements_by_class_name('summary-text')
    source_text = driver.find_elements_by_class_name('source-and-time')
        
    # 全ての要素が取得できた場合、処理を進める
    while bool(len(trends_keywords) == 0 or  len(summary_text) == 0 or len(source_text) == 0):
        time.sleep(10)
        trends_keywords = driver.find_elements_by_class_name('title')
        summary_text = driver.find_elements_by_class_name('summary-text')
        source_text = driver.find_elements_by_class_name('source-and-time')
    else:
        trends_keyword = []
        trends_abst = []
        inyo_url = []
        inyo_site = []
        for i in range(len(trends_keywords)):
            trends_keyword.append(trends_keywords[i].text)
            trends_abst.append(summary_text[i].text)
            inyo_url.append(summary_text[i].find_element_by_tag_name("a").get_attribute("href"))
            inyo_site.append(source_text[i].text.split()[0])
    
    # 各トレンドごとに詳細情報を取得する為、要素をクリックし必要なHTMLを表示させる（その為のクリックする要素を取得している）
    trend_click = driver.find_elements_by_class_name('feed-item')
    
    # 蓄積用のリストを用意しておく
    google_trend_news_list = []
    google_trend_words_list = []
    
    # トレンドごとに詳細情報を取得する為、要素をクリックし必要なHTMLを表示させる
    for trend in trend_click:
        trend.click()
        
        # 1秒間、ページが開かれるのを待つ
        time.sleep(1)
        
        # 引用ニュースを取得
        google_trend_news_tmp = driver.find_elements_by_class_name('item-title')
        google_trend_news = ''
        for news in google_trend_news_tmp:
            google_trend_news = google_trend_news + news.text
            
        # 関連ワードを取得
        google_trend_words_tmp = driver.find_elements_by_class_name('list')
        google_trend_words = google_trend_words_tmp[0].text.replace(" ","")
        
        # リストに追加
        google_trend_news_list.append(google_trend_news)
        google_trend_words_list.append(google_trend_words) 
            
    driver.quit()
    
    google_trend_data = {'goole_keyword': trends_keyword, 'google_abst': trends_abst, 'google_url': inyo_url, 'google_site': inyo_site, \
                        'google_news':google_trend_news_list, 'google_words':google_trend_words_list}
    
    return google_trend_data

In [11]:
sample5 = google_trends_search()

In [13]:
# googleトレンドの急上昇ワードなどのデータが取得できている
sample5

{'goole_keyword': ['バナナマン • TBSテレビ • サンドウィッチマン • 日村 勇紀',
  'ニジゲンノモリ • ドラゴンクエスト • 淡路島 • スクウェア・エニックス',
  '藤嶋健人 • 中日ドラゴンズ • 登板 • いい夫婦の日',
  '大山悠輔 • 阪神タイガース • 日本のプロ野球 • 契約更改 • キャリアハイ',
  '木村 文乃 • 徹子の部屋 • 黒柳 徹子 • テレビ朝日 • オールニッポン・ニュースネットワーク',
  '賞与 • 国家公務員',
  'Cyberpunk 2077 • CD Projekt • PlayStation 4',
  '日本女子プロゴルフ協会 • LPGA新人戦 加賀電子カップ • 全米女子オープン',
  '高橋 茂雄 • サバンナ • ドラゴン',
  'ふたご座流星群 • ふたご座',
  '中川 大志 • フジテレビジョン • TOKIOカケル • 松岡 昌宏 • 山本舞香',
  'ペ・ヨンジュン • パク・スジン • Instagram',
  '平子理沙 • Instagram',
  'スピードワゴン • 小沢一敬',
  'ケイト • カネボウ化粧品'],
 'google_abst': ['【ニュース】『バナナサンドSP』放送決定 今年の音楽シーンで最も注目されたあのアーティストが出演',
  '「ドラゴンクエスト アイランド」淡路島に2021年春オープン',
  '中日・藤嶋健人が結婚 お相手は名古屋市在住の一般女性「いい夫婦の日」11月22日に',
  '【阪神】新主将の大山悠輔「いい評価をしてもらった」 大幅アップで１億円の大台到達',
  '木村文乃：きょう「徹子の部屋」出演 女手一つで育ててくれた母への思い明かす',
  '国家公務員に65万円ボーナス 昨冬より3万4100円減',
  '『サイバーパンク2077』コンソール版向けの28GBの発売前アップデートを配信！',
  '「いい成績で終えれるように頑張りたい」プラチナ世代の安田祐香が“７秒動画”で新人戦へ抱負！',
  '若手女優・加藤小夏が出演！ドラゴンに扮したサバンナ高橋とゆる～いゲーム実況',
  '12月12日～14日 ふたご座流星群を観よう！',
  '中川大志の弱点は“虫”！フナムシが

## 必要な処理は用意できたので、ここからレコメンドを作っていく。

#### ちょっとしたデータ加工

In [14]:
google_trend_word = []
for i in sample5['goole_keyword']:
    google_trend_word.append(i.split(' • '))

In [15]:
google_trend_news = sample5['google_news']

In [16]:
google_trend_words = sample5['google_words']

#### 【NesAPIを使用】トレンドワードに基づいて、関連ニュースを取得する

In [17]:
sample_news = []
for j in google_trend_word:
    sample_news.append(keyword_news_search(search_word = j[0]))
    time.sleep(1)

In [18]:
news_text = []
for k in sample_news:
    news_text_tmp = ''
    for o in k['articles']:
        news_text_tmp += (o['title'] +","+ str(o['description']))
    news_text.append(news_text_tmp)

#### 【twitterAPIを使用】トレンドワードに基づいて、関連ツイートを取得する

In [19]:
twtter_text = []
for p in google_trend_word:
    twtter_text.append(twitter_search(search_keyword=p[0], search_count = 10))
    time.sleep(1)

In [20]:
twitter_text_list = []
for q in twtter_text:
    
    twitter_text_tmp = ''
    for i in range(len(q["statuses"])):
        twitter_text_tmp += q["statuses"][i]["text"]
        
    twitter_text_list.append(twitter_text_tmp)

#### ここまでの処理で取得できたデータを見てみる

In [22]:
# googleトレンドで急上昇中のワード一覧
google_trend_word

[['バナナマン', 'TBSテレビ', 'サンドウィッチマン', '日村 勇紀'],
 ['ニジゲンノモリ', 'ドラゴンクエスト', '淡路島', 'スクウェア・エニックス'],
 ['藤嶋健人', '中日ドラゴンズ', '登板', 'いい夫婦の日'],
 ['大山悠輔', '阪神タイガース', '日本のプロ野球', '契約更改', 'キャリアハイ'],
 ['木村 文乃', '徹子の部屋', '黒柳 徹子', 'テレビ朝日', 'オールニッポン・ニュースネットワーク'],
 ['賞与', '国家公務員'],
 ['Cyberpunk 2077', 'CD Projekt', 'PlayStation 4'],
 ['日本女子プロゴルフ協会', 'LPGA新人戦 加賀電子カップ', '全米女子オープン'],
 ['高橋 茂雄', 'サバンナ', 'ドラゴン'],
 ['ふたご座流星群', 'ふたご座'],
 ['中川 大志', 'フジテレビジョン', 'TOKIOカケル', '松岡 昌宏', '山本舞香'],
 ['ペ・ヨンジュン', 'パク・スジン', 'Instagram'],
 ['平子理沙', 'Instagram'],
 ['スピードワゴン', '小沢一敬'],
 ['ケイト', 'カネボウ化粧品']]

In [24]:
# googleトレンドに紐づいている関連ニュースタイトル
google_trend_news

['【ニュース】『バナナサンドSP』放送決定 ......『バナナサンド』ゴールデンSP決定 ゲストはバラエティ初出演のアーティスト',
 '「ドラゴンクエスト アイランド」淡路島に2021年春オープン淡路島に『ドラクエ』の新アトラクション登場！『ドラゴンクエスト アイランド ......',
 '中日・藤嶋健人が結婚 お相手は名古屋市在住の一般女性「いい夫婦の日」11月22日に【中日】藤嶋健人１１月２２日「いい夫婦の日」に一般女性と入籍発表中日藤嶋が結婚「見た瞬間にスローモーション」',
 '【阪神】新主将の大山悠輔「いい評価をしてもらった」 大幅アップで１億円の大台到達阪神大山5300万増の１億円！「まだまだいける」阪神・大山が大幅アップの１億円「来年に向けて強い気持ちになるサインの仕方ができた」',
 '木村文乃：きょう「徹子の部屋」出演 女手一つで育ててくれた母への思い明かす',
 '国家公務員に65万円ボーナス 昨冬より3万4100円減公務員に冬のボーナス 国家公務員平均約65万円 3年連続の減少国家公務員に冬のボーナス支給、３年連続減',
 '『サイバーパンク2077』コンソール版向けの28GBの発売前アップデートを配信！ナイトシティへようこそ！ オープンワールドRPG『サイバーパンク2077』ついに発売...',
 '...19歳安田祐香、白パーカーでの“最終戦”への決意表明に反響「可愛さに癒されます」第1日組み合わせ発表 ー JLPGA新人戦 加賀電子カップ',
 '若手女優・加藤小夏が出演！ドラゴンに扮したサバンナ高橋とゆる～いゲーム実況サバンナ高橋が“ドラゴン”に変身？ 世界一ゆるいゲーム実況番組スタートサバンナ高橋茂雄がドラゴンに扮して旬の女子とゲーム実況！12月ゲストは加藤小夏',
 '12月12日～14日 ふたご座流星群を観よう！週間天気来週は強い寒気が南下 今週のうちに冬支度を',
 '...中川大志、虫嫌いで父親に電話「何も聞かずに家まで来てください」中川大志は母性本能くすぐり系？虫だけじゃない「弱点」にキュンです',
 '...',
 '「ジョジョに出て来そう」「めっちゃエキゾチック」 .........',
 '「ひとりが寂しい男を演じてるだけ」スピードワゴン・小沢さんの恋愛論 ...「友達がいない自分」選んでない？スピードワゴン・

In [26]:
# googleトレンドに紐づいている関連ワード
google_trend_words

['fns歌謡祭第2夜\n大井競馬場\n中村仁美\n松浦亜弥橘慶太\n岡添麻希\n永島優美\n滝菜月\n鷲見玲奈インスタ\nバナナサンド\n市立船橋\nアクセサリーボックス\nあびる優現在\n椿鬼奴\nさまーず\n竹山\nさまーず大竹\n大竹一樹\nサマーズ大竹\n中村仁美大竹\n中村仁美旦那\nfns歌謡祭司会\n大竹\nさまぁず大竹\nfns司会',
 'ドラクエタクト\n淡路島\nニジゲンノモリ\nドラクエスーパーライト\nドラクエタクトリセマラ\n淡路島ドラクエ\n堀井雄二\nロールシャッハ\nニンゲンノモリ\nドラクエ淡路島\nゾーマ\n淡路島公園\n淡路島何県\n淡路島ニジゲンノモリ\nドラクエスーパーライトらいなま\n淡路島パソナ\nにじげんのもり',
 '香川コロナ\n梶谷隆幸\n鶴竜\n藤嶋健人\n帝京高校\n梶浦郁乃\n藤嶋健人結婚\n市船橋コロナ\n毛頭ない\n藤島健人\n中日藤嶋',
 '斎藤佑樹\n梶谷隆幸\nロハスジュニア\nロハス\nロハスjr\nメルロハスジュニア\n糸井嘉男\n鈴木誠也\nアルカンタラ\nロハス阪神\n阪神ロハス\n巨人契約更改\n斎藤佑樹年俸\nサンズ\n大山悠輔\nアルカンタラ阪神\nロサリオ\n中谷将大\n増田大輝\n阪神契約更改\nボーア\n菊池涼介\n糸井\n鈴木誠也年俸\nプロ野球年俸',
 '松浦亜弥\n木村文乃\nakb\nakbコロナ\n木村文乃離婚\nモンサンミッシェル\n林修の今でしょ講座\n木村文乃父親\nスーパーjチャンネル\n木村文乃子供\n木村文乃結婚\nアップフロント\n徹子の部屋今日\n徹子の部屋再放送',
 'ボーナス\n公務員ボーナス\n冬のボーナス\nボーナス平均\n国家公務員ボーナス\n賞与\n国家公務員\n地方公務員ボーナス\n冬のボーナス平均\nボーナス手取り計算\n公務員ボーナス日\n冬ボーナス\nボーナスとは\n2020冬ボーナス\nボーナス手取り\n国家公務員とは\n人事院勧告\nボーナス2020冬\n冬のボーナス2020\nボーナス公務員\n勤勉手当\nコロナボーナス\n自衛隊ボーナス\n勤勉手当とは\n人事院勧告2020',
 'サイバーパンク2077\nサイバーパンク\ncyberpunk2077\noutward\nサイバーパンク2077ps5\nサイバーパ

In [27]:
#　ちょっとデータ加工
google_trend_word_text = []
for r in google_trend_word:
    google_trend_word_text_tmp = ','.join(r)
    google_trend_word_text.append(google_trend_word_text_tmp)

#### googleトレンドのページに記載されているテキストは重みをつける（✖️3）

In [30]:
test_text = []
for s in range(len(google_trend_word_text)):
    test_text.append((google_trend_word_text[s] + google_trend_news[s] + google_trend_words[s]) * 3 + news_text[s] + twitter_text_list[s])

## 楽天APIで取得済みの商品リストを持ってくる

#### 楽天APIで書籍情報を取得し続ける処理は別途作成し実行ずみ。.pickleのファイルにしているので、ここで読み込み使用する。

In [31]:
import pickle
import pandas as pd

In [32]:
# .pickleから著者リストを読み出し
with open('book_list_csv.pickle','rb') as f:
    book_list = pickle.load(f)

In [34]:
test_books = book_list['title']+","+book_list['subTitle']+","+book_list['itemCaption']

In [36]:
df1 = pd.DataFrame()
df1['test'] = test_books

In [37]:
df2 = pd.DataFrame(test_text, columns=['test'])

In [38]:
df3 = df1.append(df2).reset_index()

#### 自然言語処理で、テキストをベクトルに変換していく

In [39]:
from sklearn import preprocessing

In [40]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [41]:
import MeCab

In [42]:
import re

In [43]:
m = MeCab.Tagger ("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd")

In [44]:
# あまり関係のないと思われる数字を全て0に置き換える関数
def replace_number_to_zero(text):
    changed_text = re.sub(r'[0-9]+', "0", text) #半角
    changed_text = re.sub(r'[０-９]+', "0", changed_text) #全角
    return changed_text

# 数字を0に置換
df3['review_number_to_zero'] = df3['test'].map(replace_number_to_zero)

In [45]:
# 分かち書きした結果を返す関数
def leaving_space_between_words_column(text):
    
    nouns = []
    
    for line in m.parse(text).splitlines():
        if len(line.split()) > 0:
            if "名詞" in line.split()[-1]:
                nouns.append(line)
    
#     nouns = [line for line in m.parse(text).splitlines()
#                if "名詞" in line.split()[-1]]

    meishi_list = []
    for str in nouns:
        meishi_list.append(str.split()[2])
    
#     return ','.join(meishi_list)
    return ' '.join(meishi_list)

# 分かち書きしたカラムをdfに追加する
df3['lsbw'] = df3['review_number_to_zero'].map(leaving_space_between_words_column)

In [46]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df3['lsbw'])

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
trend_target = X[182482:]
trend_items = X[:182481]

In [52]:
base = 182482
for u in trend_target:
    similarity = cosine_similarity(u, trend_items)
    aaa = similarity[0].tolist()
    print(df3.iloc[base, :])
    print('-----')
    for v in range(-1,-4,-1):
        max_value = sorted(aaa)[v]
        max_index = aaa.index(max_value)
        print(max_value)
        print(df3.iloc[max_index, :])
        print('-----')
    print('-----')
    print('-----')
    print('-----')
    base = base + 1

index                                                                    0
test                     バナナマン,TBSテレビ,サンドウィッチマン,日村 勇紀【ニュース】『バナナサンドSP』放送...
review_number_to_zero    バナナマン,TBSテレビ,サンドウィッチマン,日村 勇紀【ニュース】『バナナサンドSP』放送...
lsbw                     バナナマン TBSテレビ サンドウィッチマン 日村 勇紀 ニュース バナナサンド SP 放送...
Name: 182482, dtype: object
-----
0.21505204446891682
index                                 21729
test                     ぼくがうまれたひ,大竹典子童謡詩集,
review_number_to_zero    ぼくがうまれたひ,大竹典子童謡詩集,
lsbw                      ぼく 躱避 大竹 典子 童謡 詩集
Name: 21729, dtype: object
-----
0.13942551461594022
index                          16265
test                     愛しの桜さん（2）,,
review_number_to_zero    愛しの桜さん（0）,,
lsbw                          桜 さん 0
Name: 16265, dtype: object
-----
0.13942551461594022
index                          16265
test                     愛しの桜さん（2）,,
review_number_to_zero    愛しの桜さん（0）,,
lsbw                          桜 さん 0
Name: 16265, dtype: object
-----
-----
-----
-----
index            

index                                                                    8
test                     高橋 茂雄,サバンナ,ドラゴン若手女優・加藤小夏が出演！ドラゴンに扮したサバンナ高橋とゆる～...
review_number_to_zero    高橋 茂雄,サバンナ,ドラゴン若手女優・加藤小夏が出演！ドラゴンに扮したサバンナ高橋とゆる～...
lsbw                     高橋 茂雄 サバンナ ドラゴン 若手女優 加藤小夏 出演 ドラゴン 扮 サバンナ高橋 ゲーム...
Name: 182490, dtype: object
-----
0.2539877811406617
index                         23988
test                     赤い目のドラゴン,,
review_number_to_zero    赤い目のドラゴン,,
lsbw                         目 ドラゴン
Name: 23988, dtype: object
-----
0.19636625978898611
index                                                                18637
test                     ドラゴンはキャプテン,,たいへん！ドラゴンがさらわれて、「かえしてほしければ、雨をふらすじゅ...
review_number_to_zero    ドラゴンはキャプテン,,たいへん！ドラゴンがさらわれて、「かえしてほしければ、雨をふらすじゅ...
lsbw                     ドラゴン キャプテン ドラゴン 雨 じゅもん でんわ だれ ドラゴン どこ なかよし きい ...
Name: 18637, dtype: object
-----
0.19293382864516478
index                                                                 7088
test                  

## 現状の課題

- 商品総数が10万件しかない為、微妙なコンテンツがヒットしやすい<br>→ 楽天から取得する商品数は100万件ぐらいまでは増やすことができるので、商品数を増やして結果を見てみる
- アブストが入っていない商品が多く、情報量が少ない<br>→ 商品数増やして何とかなるならいいが・・・。誤抽出しても問題ないような見せ方が必要